In [1]:
import os
import sys

In [11]:
%pwd

'g:\\My Drive\\Text Summarizer'

In [3]:
os.chdir("../")

In [14]:
sys.path.append(os.path.abspath('G:/My Drive/Text Summarizer/src'))

In [15]:
from textsummarizer.constants import *

In [21]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_url: str
    local_data_file: Path
    unzip_dir: Path

In [22]:
from textsummarizer.utils.common import read_yaml, create_directories

In [23]:
class Configurationmanager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH):
        
            self.config = read_yaml(config_filepath)
            self.params = read_yaml(params_filepath)

            create_directories([self.config.artifacts_root])

    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
          config = self.config.data_ingestion

          create_directories([config.root_dir])

          data_ingestion_config = DataIngestionConfig(

                root_dir=config.root_dir,
                source_url=config.source_url,
                local_data_file=config.local_data_file,
                unzip_dir=config.unzip_dir
          )

          return data_ingestion_config

In [24]:
import urllib.request as request
import zipfile
from textsummarizer.loggin import logger
from textsummarizer.utils.common import get_size

In [27]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config



    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_url, 
                filename = self.config.local_data_file
            )
            logger.info(f" {filename} download! with following info:\n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file)  )}")



    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts zip file to the data directory
        Function returns None

        """

        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

try:
    config = Configurationmanager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()

except Exception as e:
    raise e

[2024-07-11 12:51:19,271: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-07-11 12:51:19,285: INFO: common: yaml file: params.yaml loaded successfully]


[2024-07-11 12:51:19,335: INFO: common: Created directory: artifacts]
[2024-07-11 12:51:19,621: INFO: common: Created directory: artifacts/data_ingestion]
[2024-07-11 12:51:24,840: INFO: 1877138409:  artifacts/data_ingestion/data.zip download! with following info:
Connection: close
Content-Length: 7903594
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "dbc016a060da18070593b83afff580c9b300f0b6ea4147a7988433e04df246ca"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 3FBD:FD0B7:446A3:509EC:668FC73B
Accept-Ranges: bytes
Date: Thu, 11 Jul 2024 11:51:24 GMT
Via: 1.1 varnish
X-Served-By: cache-man4149-MAN
X-Cache: MISS
X-Cache-Hits: 0
X-Timer: S1720698684.888428,VS0,VE564
Vary: Authorization,Accept-Encoding,Origin
Access-Control-Allow-Origin: *
Cross-Origin-Resource-Policy: cross-origin
X-Fastly-R